## 数据预处理

In [1]:
import pandas as pd
import xlrd
xlsfile = "B_vol2.xlsx"
book = xlrd.open_workbook(xlsfile)
table = book.sheet_by_index(0)
nrows = table.nrows
ncols = table.ncols
print("The table all row is %d,col is %s" %(nrows,ncols))

The table all row is 2600,col is 1


In [2]:
data = []
for i in range(nrows):
    data.append(table.cell(i,0).value)

In [3]:
data_rev = list(reversed(data))

## 处理掉断行的题目

In [4]:
import re
m = re.compile('(\d+\u3001)|([A-D]\.)')
f = [re.search(m,data_rev[i]) for i in range(len(data_rev))]

In [5]:
for i in range(len(f)-1):
    if f[i] is None:
        data_rev[i+1] = '%s%s' % (data_rev[i+1], data_rev[i])
        data_rev[i] = 'flag'

In [6]:
for i in range(len(data_rev)-1,-1,-1):
    if data_rev[i] == 'flag':
        data_rev.pop(i)
data = list(reversed(data_rev))

## 匹配ABCD选项

In [7]:
def selet_div(data):
    '''
    
    对data中alph选项（ABCD）做筛选
    
    '''
    s_num = '\D*((?:\d)+)(\u3001)\s*(.+)'    #匹配题号
    colnum2qnum = [None] * len(data)    #行数和题号
    colnum2qnum[0] = 1
    qnum_que = [[],[]]    # 题号和问题
    qnum_que[0].append(re.search(s_num, data[0]).group(1))
    qnum_que[1].append(re.search(s_num, data[0]).group(3))
    for i in range(1,len(data)):
        m = re.search(s_num, data[i])
        if m is None:
            colnum2qnum[i] = colnum2qnum[i-1]
        else:
            qnum_que[0].append(m.group(1))
            qnum_que[1].append(m.group(3))
            colnum2qnum[i] = int((m.group(1)))

    #pat_slt = '([A-D]\.[\u4e00-\u9fa5]+)\s+(([A-D]\.[\u4e00-\u9fa5]+)?)'    #匹配ABCD选项  A.XXXX    B.XXXX
    # 匹配四个选项
    #pat_slt = '[^\w\u4e00-\u9fa5]*([A-D]\..*[\w\d\u4e00-\u9fa5]+)\s*([A-D]\.\s*\S+)?'
    pat_slt = '([A-Z]\.\S+)\s*([A-Z]\.\S+)?'
    form = [[],[]] # quesion num, select
    for i in range(len(data)):
        m = re.search(pat_slt, data[i])
        if m is not None:
            k = list(m.groups())
            if None in k: k.remove(None)
            form[0].extend([colnum2qnum[i]]*len(k))
            form[1].extend(k)
    
    final = [[],[],[]]
    final[0] = qnum_que[0]
    final[1] = qnum_que[1]
    df = pd.DataFrame({'key':form[0],'data':form[1]})
    grouped=df['data'].groupby(df['key'])
    gr = []
    for name,group in grouped:
        final[2].append('$'.join(list(grouped.get_group(name))))
    
    return final

上述函数需要再加一个返回题号的功能

In [8]:
f = selet_div(data)

In [9]:
import xlwt
book = xlwt.Workbook(encoding='utf-8')
sheet1 = book.add_sheet('test')
for i in range(len(f[0])):
    for j in range(3):
        sheet1.write(i,j,f[j][i])
book.save(r'final.xls')